In [227]:
import requests
import numpy as np
from math import log, exp
from scipy.optimize import linear_sum_assignment
import time

In [228]:
headers = {
        "X-RapidAPI-Key": "646b47ee09mshb839bd4c8a546a0p121e42jsn90bf68a43620",
        "X-RapidAPI-Host": "twelve-data1.p.rapidapi.com"
    }

url = "https://twelve-data1.p.rapidapi.com/price"

def get_price(curr1, curr2):
    if curr1 == curr2:
        return 1
    else:
        try:
            return float(eval(requests.request("GET", url, headers=headers, params={"symbol":f"{curr1}/{curr2}","format":"json","outputsize":"30"}).text)['price'])
        except KeyError:
            return 0

In [229]:
def forex_trade(curr = np.array(['AUD', 'CHF', 'EUR', 'JPY'])):

    M = np.matrix([[0.0 for _ in curr]for _ in curr], dtype=float)
    C = np.matrix([[False for _ in curr]for _ in curr], dtype=bool)

    for r, curr1 in enumerate(curr):
        for c, curr2 in enumerate(curr):
            if C[r, c] == False:
                p = get_price(curr1, curr2)
                if p != 0:
                    M[r, c] = log(float(p))
                    M[c, r] = log(float(1/p))
                else:
                    M[r, c] = 0.0
                    M[c, r] = 0.0
                C[r, c] = C[c, r] = True
        
    
    row_ind, col_ind = linear_sum_assignment(M, maximize=True)

    return M, row_ind, col_ind

In [233]:
while True:
    M, row_ind, col_ind = forex_trade()
    realized_ratio = exp(M[row_ind, col_ind].sum())

    print(realized_ratio)

    time.sleep(31)

1.001426628856065
1.4916467780429594
1.0
1.0
1.0
1.0
1.4362244531575397
1.4362244531575397
1.4916467780429594
1.4916467780429594
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.4362244531575397
1.4362244531575397


KeyboardInterrupt: 

In [237]:
M, row_ind, col_ind = forex_trade()
M

matrix([[ 0.        , -0.36201776, -0.39988073,  4.55175886],
        [ 0.36201776,  0.        , -0.03926071,  4.91272841],
        [ 0.39988073,  0.03926071,  0.        ,  4.95201699],
        [-4.55175886, -4.91272841, -4.95201699,  0.        ]])